In [ ]:
#Download Blender from Repository - Run Once unless version changed

!wget https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz

In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Copy Blender to gdrive - Run Once unless version changed
!cp /content/blender-4.3.0-linux-x64.tar.xz /content/drive/MyDrive/Blender/blender-4.3.0-linux-x64.tar.xz

In [ ]:
#Copy Blender back to Colab
!cp /content/drive/MyDrive/Blender/blender-4.3.0-linux-x64.tar.xz /content/blender-4.3.0-linux-x64.tar.xz

In [ ]:
#install Blender
!tar xf blender-4.4.0-linux-x64.tar.xz

In [ ]:
# 1. Descargar el complemento PCB2Blender
!wget -O pcb2blender_importer.zip "https://github.com/30350n/pcb2blender/releases/download/v2.17.2-k9.0-b4.2lts/pcb2blender_importer_v2-17-2_b4-4.zip"
# 2. Descomprimir el archivo descargado
!unzip -o pcb2blender_importer.zip -d /tmp/pcb2blender_importer

# 3. Instalar la dependencia `error_helper` (corrigiendo la versión)
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install /tmp/pcb2blender_importer/wheels/error_helper-*.whl

# Instalar Skia en el entorno de Python de Blender
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install skia-python

# 4. Crear el directorio de addons en Blender
!mkdir -p ~/.config/blender/4.4/scripts/addons/

# 5. Copiar el complemento al directorio de Blender
!cp -r /tmp/pcb2blender_importer ~/.config/blender/4.4/scripts/addons/pcb2blender

# 6. Crear un script para habilitar el addon automáticamente
!echo "import bpy; bpy.ops.preferences.addon_enable(module='pcb2blender'); bpy.ops.wm.save_userpref()" > enable_addon.py

# 7. Instalar la librería Pillow (requerida por el complemento)
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install Pillow

# 8. Ejecutar Blender en segundo plano para activar el addon
!./blender-4.4.0-linux-x64/blender --background --python enable_addon.py

# 9. Eliminar el script auxiliar después de habilitar el addon
!rm enable_addon.py

In [ ]:
#patch init_py bl_info missing
init_file = "/root/.config/blender/4.4/scripts/addons/pcb2blender/__init__.py"

bl_info_block = '''bl_info = {
    "name": "PCB2Blender",
    "author": "30350n",
    "version": (2, 17, 2),
    "blender": (4, 4, 0),
    "location": "File > Import",
    "description": "Import PCB data into Blender",
    "category": "Import-Export",
}

'''

with open(init_file, 'r') as f:
    content = f.read()

if "bl_info" not in content:
    print("Agregando bl_info al inicio del archivo...")
    with open(init_file, 'w') as f:
        f.write(bl_info_block + content)
else:
    print("bl_info ya está presente. No se hacen cambios.")


In [ ]:
%%writefile list_addons.py
import addon_utils
for mod in addon_utils.modules():
    print(mod.__name__)

In [ ]:
!./blender-4.4.0-linux-x64/blender -b -P list_addons.py

In [ ]:
#set the paths to Blender Files
filename = '/content/drive/MyDrive/Blender/test.blend'

In [ ]:
#Render
!./blender-4.4.0-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o '/content/drive/MyDrive/Blender/Test1/Renders/testRender_'  -s 1 -e 2 -a -F 'PNG' -- --cycles-device CUDA

In [ ]:
!./blender-4.4.0-linux-x64/blender -b "$filename" --gpu-backend opengl --gpu-compilation-subprocesses 2 -E 'CYCLES' -o '/content/drive/MyDrive/Blender/Test1/Renders/testRender' -s 150 -e 175 -a -F 'FFMPEG' --render-format 'MPEG4' --ffmpeg-format 'MPEG4' --ffmpeg-video-bitrate 8000 --ffmpeg-audio-codec 'NONE' -- --cycles-device OPTIX+CPU --cycles-print-stats -t $(nproc)

In [ ]:
!./blender-4.4.0-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o '/content/drive/MyDrive/Blender/Test1/Renders/testRender' -s 50 -e 75 -a -F 'FFMPEG' -- --cycles-device OPTIX --cycles-print-stats -t 50